# ML Graph Embedding methods

In [1]:
#import libraries
import torch
import time
from torch_geometric.nn import Node2Vec

In [2]:
# Load the graph from the file
server_graph_data_filepath = './data/server_graph_data.pth'
graph_data = torch.load(server_graph_data_filepath)

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
print(device)
graph_data = graph_data.to(device)

cuda


In [ ]:
# Initialize the Node2Vec model
node2vec = Node2Vec(
    graph_data.edge_index,       # Edge list
    embedding_dim=3,      # Size of embeddings
    walk_length=20,        # Length of each random walk
    context_size=10,       # Window size for Skip-Gram
    walks_per_node=10,     # Number of walks per node
    num_negative_samples=1,  # Number of negative samples for Skip-Gram
    p=0.25,  # Return parameter: encourages staying close to the starting node
    q=4.0,   # In-out parameter: encourages exploring further away
    sparse=True            # Use sparse gradients for efficiency
).to(device)

# Define the optimizer
optimizer = torch.optim.SparseAdam(list(node2vec.parameters()), lr=0.01)

# Training loop
def train():
    node2vec.train()
    total_loss = 0
    loader = node2vec.loader(batch_size=8, shuffle=True)
    for pos_rw, neg_rw in loader:
        pos_rw = pos_rw.to(device)
        neg_rw = neg_rw.to(device)
        
        optimizer.zero_grad()
        loss = node2vec.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Run training for multiple epochs
for epoch in range(1, 101):
    # Save timestamp
    start = time.time()
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
    # Save timestamp
    end = time.time()
    
    print(end - start)

# Obtain the node embeddings
node_embeddings = node2vec.embedding.weight.data

print("Node Embeddings Shape:", node_embeddings.shape)


In [ ]:
node_embeddings

In [ ]:
node2vector_embedding_filepath = 'node2vector_embeddings.pt'
torch.save(node_embeddings, node2vector_embedding_filepath)

In [ ]:
# Initialize the DeepWalk model by setting p and q to 1 (unbiased random walks)
deepwalk = Node2Vec(
    graph_data.edge_index,
    embedding_dim=64,
    walk_length=40,        # Longer walk length for DeepWalk
    context_size=10,
    walks_per_node=10,
    sparse=True
).to(device)

# Define the optimizer
optimizer = torch.optim.SparseAdam(list(deepwalk.parameters()), lr=0.01)

# Training loop
def train():
    deepwalk.train()
    total_loss = 0
    loader = deepwalk.loader(batch_size=8, shuffle=True)
    for pos_rw, neg_rw in loader:
        pos_rw = pos_rw.to(device)
        neg_rw = neg_rw.to(device)
        
        optimizer.zero_grad()
        loss = deepwalk.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)

# Run training for multiple epochs
for epoch in range(1, 101):
    start = time.time()
    loss = train()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}')
        
    # Save timestamp
    end = time.time()
    
    print(end - start)

# Obtain the node embeddings
node_embeddings_deepwalk = deepwalk.embedding.weight.data

print("Node Embeddings Shape:", node_embeddings.shape)


In [ ]:
node_embeddings_deepwalk

In [ ]:
deepwalk_embedding_filepath = 'deepwalk_embeddings.pt'
torch.save(node_embeddings, deepwalk_embedding_filepath)